In [30]:
from langchain.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
import os


In [31]:
load_dotenv() 
api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key

In [32]:
loader = PyPDFLoader("documents/ctse.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

In [33]:
embedding = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(split_docs, embedding)
retriever = vectorstore.as_retriever()

In [34]:
prompt_template = ChatPromptTemplate.from_template("""
You are an intelligent assistant helping software engineering students understand their lecture material.

Use the following retrieved lecture content to answer the user's question clearly, concisely, and helpfully. If the context is not sufficient to answer the question, say so honestly.

Retrieved Lecture Context:
{context}

Student Question:
{question}

Answer:
""")

In [35]:
llm = OpenAI(temperature=0.7)
document_chain = create_stuff_documents_chain(llm, prompt_template)


In [36]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt_template}
)

In [37]:

while True:
    user_query = input("You: ")
    if user_query.lower() == "exit":
        print("Goodbye!")
        break
    

    response = rag_chain({"query": user_query})


    

    if isinstance(response, dict):

        result = response.get("result", response.get("answer", response.get("output", str(response))))
        print(f"Bot: {result}\n")
    else:
        print(f"Bot: {response}\n")

Bot: The main point of implementing DevOps practices is to improve collaboration and communication between developers and operators. It aims to break down organizational silos and promote shared ownership of production. Other key aspects include measuring everything, implementing gradual changes, accepting failure as normal, and leveraging tooling and automation to increase efficiency and productivity. Ultimately, the goal of DevOps is to achieve agility and stability in software development and operations.

Goodbye!
